In [1]:
import pandas as pd
import torch
import lightning as L

from model.modeling_demolta import DeMOLTaConfig
from trainer import LitMOLAForRegression, SaveTrainableParamsCheckpoint
from datautils import LitMOLAFintTuneDataModule

In [2]:
BATCH_SIZE = 4
SEED = 42
TEXT_MODEL_NAME = './Llama-2-7b-hf'

In [3]:
L.seed_everything(SEED)

Global seed set to 42


42

In [4]:
lit_finetune_data_module = LitMOLAFintTuneDataModule(
    df_path='./data/train.csv',
    batch_size=BATCH_SIZE,
    seed=SEED,
    k_fold=5,
    train_fold=0,
)

In [5]:
demolta_config = DeMOLTaConfig(
    num_layers=12,
    node_hidden_dim=768,
    edge_hidden_dim=256,
    node_ff_dim=3072,
    edge_ff_dim=1536,
    num_heads=12,
)

In [6]:
lit_model = LitMOLAForRegression(
    demolta_config=demolta_config,
    text_model_name=TEXT_MODEL_NAME,
    n_class=2,
)

In [7]:
checkpoint = torch.load('./checkpoints/mola-pretrain-base-Llama-2-7b-hf-step=60000-train_loss=2.1271-val_loss=2.60.ckpt')
lit_model.load_state_dict(checkpoint, strict=False)

_IncompatibleKeys(missing_keys=['model.regressor.1.weight', 'model.regressor.1.bias', 'model.regressor.2.weight', 'model.regressor.2.bias', 'model.regressor.5.weight', 'model.regressor.5.bias'], unexpected_keys=['model.language_projection.weight', 'model.language_projection.bias'])

In [8]:
# for param in lit_model.model.mol_model.parameters():
#     param.requires_grad = False

In [9]:
checkpoint_callback = SaveTrainableParamsCheckpoint(
    monitor='val_loss',
    dirpath='./checkpoints/',
    filename='mola-pretrain=60000-finetune-{epoch}-{val_loss:.4f}',
    save_top_k=1,
)

In [10]:
trainer = L.Trainer(
    accelerator='gpu',
    max_epochs=10,
    callbacks=[checkpoint_callback],
    val_check_interval=0.5,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
trainer.fit(lit_model, lit_finetune_data_module)

You are using a CUDA device ('NVIDIA GeForce RTX 3070') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Skipped loading some Jax models, missing a dependency. No module named 'jax'
c:\Users\dust\Documents\Dacon\med\demolta\datautils.py:142: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby('SMILES').mean().reset_index()
c:\Users\dust\an

Sanity Checking: 0it [00:00, ?it/s]

c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
